In [1]:
import os
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
path = 'classifier_cv_results/'
file_names = os.listdir(path)
[file_name for file_name in file_names if 'labels' in file_name]

['Vegas - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Isla Vista - All Excerpts - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Marysville - All Excerpts - Final - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Newtown - All Excerpts - 1-2-2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Orlando - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Charleston - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Vegas - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Isla Vista - All Excerpts - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Charleston - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Orlando - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'San Bernardino - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv']

## Sentence Based Results For Each Event

In [3]:
sentence_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' in file_name and 'labels' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        sentence_results_df = pd.concat([sentence_results_df, event_df], ignore_index=True)

# get mean of cross val
df = sentence_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 
                                 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean().reset_index()



Vegas - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Isla Vista - All Excerpts - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Orlando - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Charleston - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv


In [4]:
# get max of each classification method
fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label', 'event']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        fmax_df = fmax_df.append(fmax_row.loc[0,:], ignore_index = True)
        
fmax_df

,classifier,event,fscore,index,label,num_test_exs,num_train_exs,precision,recall,vectorizer
0,svm_balanced,Charleston,0.398347,19.0,ACCOUNT,57.333333,210.666667,0.316495,0.537793,cust_all-3gram
1,svm_balanced,Isla Vista,0.767881,47.0,ACCOUNT,366.000000,1475.000000,0.714171,0.830418,char
2,svm_balanced,Orlando,0.444496,103.0,ACCOUNT,62.000000,253.000000,0.338007,0.650618,cust_only_alpha-1gram
3,logregcv_balanced,Vegas,0.280416,117.0,ACCOUNT,26.000000,92.000000,0.430403,0.210317,char
4,logregcv_balanced,Charleston,0.628657,153.0,EVENT,99.333333,413.666667,0.597270,0.664036,char
5,logregcv_balanced,Isla Vista,0.755741,189.0,EVENT,294.333333,1154.666667,0.729246,0.784463,char
6,logregcv_balanced,Orlando,0.666639,229.0,EVENT,81.000000,326.000000,0.636454,0.705084,cust_all-1gram
7,logregcv_balanced,Vegas,0.703499,261.0,EVENT,75.666667,307.333333,0.691133,0.717256,char
8,svm_balanced,Charleston,0.322888,319.0,GRIEF,20.666667,101.333333,0.229781,0.545238,cust_only_alpha-1gram
9,logregcv,Isla Vista,0.801695,332.0,GRIEF,152.000000,577.000000,0.861314,0.750255,char


In [5]:
df = fmax_df.loc[:, ['label', 'event', 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]


labels = ['ALL']+list(set(df['label']))
events = ['ALL']+list(set(df['event']))
@interact
def show_results(column1='event', column2='label',
                 column3='num_test_exs', column4='num_train_exs',
                 event=events, label = labels, 
                 num_test = (0,50,1), num_train = (0,100,1)):
    
    df_len = df.shape[0]
    num_test_filt = df[column3] > num_test
    num_train_filt = df[column4] > num_train
    event_filt =[(df.loc[i, column1] == event) or (event == 'ALL') for i in range(df_len)]
    label_filt =[(df.loc[i, column2] == label) or (label == 'ALL') for i in range(df_len)]
    final_filt = [num_test_filt[i]&num_train_filt[i]&event_filt[i]&label_filt[i] for i in range(df_len)]
    
    return df.loc[final_filt]

interactive(children=(Text(value='event', description='column1'), Text(value='label', description='column2'), …

In [8]:
from pivottablejs import pivot_ui
pivot_ui(df,outfile_path='pivottablejs.html')
#HTML(‘pivottablejs.html’)

## Excerpt Based Results for Each Event

In [ ]:


excerpt_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' not in file_name and 'labels' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        excerpt_results_df = pd.concat([excerpt_results_df, event_df], ignore_index=True)

df = excerpt_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 
                                'recall', 'precision', 'fscore',
                                'num_test_exs', 'num_train_exs']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()

In [ ]:
# get max of each classification method
excerpt_fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label', 'event']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        fmax_df = fmax_df.append(fmax_row.loc[0,:], ignore_index = True)
        
excerpt_fmax_df

df_ex = df_cv_mean.reset_index().groupby(['label', 
                                     'event']).max().reset_index().loc[:, ['fscore', 'recall', 'precision', 
                                                                           'label', 'event',
                                                                            'num_test_exs', 'num_train_exs']]
df_ex

In [ ]:
df_ex = df_cv_mean.reset_index().groupby(['label', 
                                     'event']).max().reset_index().loc[:, ['fscore', 'recall', 'precision', 
                                                                           'label', 'event',
                                                                            'num_test_exs', 'num_train_exs']]
labels = ['ALL']+list(set(df_ex['label']))
events = ['ALL']+list(set(df_ex['event']))
@interact
def show_results(column1='event', column2='label',
                 column3='num_test_exs', column4='num_train_exs',
                 event=events, label = labels, 
                 num_test = (0,50,1), num_train = (0,100,1)):
    
    df_len = df_ex.shape[0]
    num_test_filt = df_ex[column3] > num_test
    num_train_filt = df_ex[column4] > num_train
    event_filt =[(df_ex.loc[i, column1] == event) or (event == 'ALL') for i in range(df_len)]
    label_filt =[(df_ex.loc[i, column2] == label) or (label == 'ALL') for i in range(df_len)]
    final_filt = [num_test_filt[i]&num_train_filt[i]&event_filt[i]&label_filt[i] for i in range(df_len)]
    
    return df_ex.loc[final_filt]

## All Events


In [ ]:
### Sentence Based

sentence_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' in file_name and 'labels' in file_name and 'fulldata' in file_name:
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        sentence_results_df = pd.concat([sentence_results_df, event_df], ignore_index=True)

df = sentence_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()
df_results = df_cv_mean.reset_index().groupby(['label', 
                                     'event']).max().reset_index().loc[:, ['fscore', 'recall', 
                                                                            'precision', 'label', 'event']]
df_results

In [ ]:
### Event Based

event_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' not in file_name and 'labels' in file_name and 'fulldata' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)      
        event_results_df = pd.concat([event_results_df, event_df], ignore_index=True)

df = event_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()
df_results = df_cv_mean.reset_index().groupby(['label', 
                                     'event']).max().reset_index().loc[:, ['fscore', 'recall', 
                                                                            'precision', 'label', 'event']]
df_results